# 0. Package, Library

In [1]:
from pybaseball import statcast, cache
cache.enable()  # 캐시 사용 → 반복 호출 방지

import pandas as pd
import numpy as np

from IPython.display import display, HTML
from datetime import datetime, timedelta
import os
import json

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid", color_codes=True)
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')  # 시각화 시 경고 메시지 제거
import shap

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, brier_score_loss, roc_auc_score, roc_curve
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from scipy.stats import uniform, randint, mode
from xgboost import XGBClassifier
from xgboost import plot_importance
from lightgbm import LGBMClassifier

In [2]:
# display option
pd.reset_option("all")

In [ ]:
# EDITOR_MODE (vsc, colab) selection
EDITOR_MODE = 'vsc'

if EDITOR_MODE == 'colab':
    from google.colab import drive
    drive.mount('/content/drive')

    # Colab 환경에서는 쉘 명령어로 pip 설치 가능
    !pip install -q pybaseball
    !pip install -q pyarrow
    !pip install -q xgboost
    !pip install -q lightgbm
    
elif EDITOR_MODE == 'vsc':
    print("VSC EDITOR_MODE")

VSC EDITOR_MODE


# 1. Data Loader

In [4]:
# API에서 데이터 받아서 오늘 날짜의 데이터를 fv에 concat 한 후 parquet으로 저장

In [5]:
COLAB_PATH = '/content/drive/MyDrive/'
VSC_PATH = '../data_v1/'  # 상위 폴더

DATA_PATH_FV = 'fv.parquet'

# fv
if EDITOR_MODE == 'colab':
    file_path = COLAB_PATH + DATA_PATH_FV
else:
    file_path = VSC_PATH + DATA_PATH_FV

print("df_fv")
df_fv = pd.read_parquet(file_path)
df_fv



df_fv


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle,attack_angle,attack_direction,swing_path_tilt,intercept_ball_minus_batter_pos_x_inches,intercept_ball_minus_batter_pos_y_inches,season
0,FF,2025-06-25,102.5,-1.71,6.03,"Miller, Mason",672761,695243,field_out,hit_into_play,...,0.92,0.74,-0.74,<NA>,-9.111255,13.440774,28.505519,28.146579,20.696262,2025
1,SL,2025-06-25,89.9,-1.99,5.95,"Miller, Mason",672761,695243,None,ball,...,3.12,-0.27,0.27,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2025
2,SL,2025-06-25,89.4,-1.89,5.87,"Miller, Mason",672761,695243,None,swinging_strike,...,2.96,-0.48,0.48,<NA>,18.325631,-24.433497,28.610431,36.125112,43.502268,2025
3,SL,2025-06-25,88.7,-1.85,5.9,"Miller, Mason",672761,695243,None,called_strike,...,2.95,-0.8,0.8,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2025
4,FF,2025-06-25,102.0,-1.78,5.97,"Miller, Mason",672761,695243,None,ball,...,0.77,0.91,-0.91,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97093,CH,2025-06-01,86.5,2.09,5.69,"Bubic, Kris",669234,663460,None,foul,...,3.23,0.71,-0.71,35.0,5.56438,0.721656,48.912236,24.898612,28.19408,2025
97094,SL,2025-06-01,85.2,2.22,5.7,"Bubic, Kris",669234,663460,None,swinging_strike,...,3.04,-0.13,0.13,43.7,-7.623744,29.274299,44.138254,36.216484,16.312573,2025
97095,SI,2025-06-01,91.9,2.22,5.63,"Bubic, Kris",669234,663460,None,called_strike,...,1.97,1.06,-1.06,34.6,<NA>,<NA>,<NA>,<NA>,<NA>,2025
97096,SI,2025-06-01,92.9,2.07,5.72,"Bubic, Kris",669234,663460,None,ball,...,1.88,0.94,-0.94,38.9,<NA>,<NA>,<NA>,<NA>,<NA>,2025


## Match Selection

In [6]:
# home_team, away_team에서 등장한 팀들 집합
teams = sorted(set(df_fv['home_team'].unique()) | set(df_fv['away_team'].unique()))

# 한 줄에 5개씩 출력
print("MLB 팀 목록 (총 {}개):\n".format(len(teams)))
for i in range(0, len(teams), 5):
    print(", ".join(teams[i:i+5]))

MLB 팀 목록 (총 30개):

ATH, ATL, AZ, BAL, BOS
CHC, CIN, CLE, COL, CWS
DET, HOU, KC, LAA, LAD
MIA, MIL, MIN, NYM, NYY
PHI, PIT, SD, SEA, SF
STL, TB, TEX, TOR, WSH


In [7]:
# Multiple Match

# match list
# 날짜와 매치 정보 추출
df_fv['game_date'] = pd.to_datetime(df_fv['game_date'])
df_fv['date_str'] = df_fv['game_date'].dt.strftime('%y%m%d')

# 매치 ID: "AWAY@HOME" 형태로 생성
df_fv['match_id'] = df_fv['away_team'] + "@" + df_fv['home_team']

# 날짜별 매치 집합 생성
date_match_dict = (
    df_fv.groupby('date_str')['match_id']
    .agg(lambda x: sorted(set(x)))  # 정렬된 리스트로 저장
    .to_dict()
)

'''
# 저장 경로
match_json_path = "fv_match_list.json"

# JSON으로 저장
with open(match_json_path, "w") as f:
    json.dump(date_match_dict, f, indent=2)

print(f"✅ Saved match list by date to {match_json_path}")
'''

'\n# 저장 경로\nmatch_json_path = "fv_match_list.json"\n\n# JSON으로 저장\nwith open(match_json_path, "w") as f:\n    json.dump(date_match_dict, f, indent=2)\n\nprint(f"✅ Saved match list by date to {match_json_path}")\n'

In [8]:
# 1. 날짜 컬럼 처리
df_fv["game_date"] = pd.to_datetime(df_fv["game_date"])

# 2. JSON 매치 리스트 로드
with open("fv_match_list.json", "r") as f:
    date_match_dict = json.load(f)

# 3. 결과 리스트 초기화
fv_game_rows = []

# 4. 날짜별 루프
for date_str, match_list in date_match_dict.items():
    pred_date = pd.to_datetime(date_str, format="%y%m%d")

    for match_id in match_list:
        away_team, home_team = match_id.split("@")

        # ▶ 홈팀의 예측일 이전 경기 중 가장 최근 game_pk
        df_home_all = df_fv[
            (df_fv["home_team"] == home_team) &
            (df_fv["game_date"] < pred_date)
        ]
        if not df_home_all.empty:
            latest_home_pk = df_home_all.sort_values("game_date", ascending=False)["game_pk"].iloc[0]
            df_home = df_fv[df_fv["game_pk"] == latest_home_pk]
        else:
            df_home = None

        # ▶ 어웨이팀의 예측일 이전 경기 중 가장 최근 game_pk
        df_away_all = df_fv[
            (df_fv["away_team"] == away_team) &
            (df_fv["game_date"] < pred_date)
        ]
        if not df_away_all.empty:
            latest_away_pk = df_away_all.sort_values("game_date", ascending=False)["game_pk"].iloc[0]
            df_away = df_fv[df_fv["game_pk"] == latest_away_pk]
        else:
            df_away = None

        # ▶ 예외 처리 사유 기록
        if df_home is None and df_away is None:
            reason = "home_and_away_not_found"
        elif df_home is None:
            reason = "home_not_found"
        elif df_away is None:
            reason = "away_not_found"
        else:
            reason = None

        # ▶ 결과 저장
        fv_game_rows.append({
            "date_str": date_str,
            "match_id": match_id,
            "home_team": home_team,
            "away_team": away_team,
            "df_home": df_home if df_home is not None and not df_home.empty else None,
            "df_away": df_away if df_away is not None and not df_away.empty else None,
            "reason": reason
        })


In [9]:
print(type(fv_game_rows))
print(type(fv_game_rows[0]))
print(fv_game_rows[0].keys())  # 딕셔너리 키 구조(열) 보기
print("fv_game_rows 행 개수:", len(fv_game_rows))

home_count = sum(1 for row in fv_game_rows if row["df_home"] is not None)
away_count = sum(1 for row in fv_game_rows if row["df_away"] is not None)

print(f"유효한 홈팀 경기 수: {home_count}")
print(f"유효한 어웨이팀 경기 수: {away_count}")

valid_match_count = sum(1 for row in fv_game_rows if row["df_home"] is not None and row["df_away"] is not None)
print(f"예측 가능한 경기 수 (홈+어웨이 모두 있음): {valid_match_count}")



# 시각화용 정보만 추출
preview_rows = []
for row in fv_game_rows[:5]:  # 앞 5개만 확인
    preview_rows.append({
        "date_str": row["date_str"],
        "match_id": row["match_id"],
        "home_team": row["home_team"],
        "away_team": row["away_team"],
        "home_game_date": row["df_home"]["game_date"].values[0] if row["df_home"] is not None else None,
        "away_game_date": row["df_away"]["game_date"].values[0] if row["df_away"] is not None else None,
        "reason": row["reason"]
    })

# DataFrame으로 변환 및 출력
preview_df = pd.DataFrame(preview_rows)
display(preview_df)

<class 'list'>
<class 'dict'>
dict_keys(['date_str', 'match_id', 'home_team', 'away_team', 'df_home', 'df_away', 'reason'])
fv_game_rows 행 개수: 341
유효한 홈팀 경기 수: 311
유효한 어웨이팀 경기 수: 311
예측 가능한 경기 수 (홈+어웨이 모두 있음): 304


,date_str,match_id,home_team,away_team,home_game_date,away_game_date,reason
0,250601,ATH@TOR,TOR,ATH,None,None,home_and_away_not_found
1,250601,BOS@ATL,ATL,BOS,None,None,home_and_away_not_found
2,250601,CIN@CHC,CHC,CIN,None,None,home_and_away_not_found
3,250601,COL@NYM,NYM,COL,None,None,home_and_away_not_found
4,250601,CWS@BAL,BAL,CWS,None,None,home_and_away_not_found


In [ ]:
# fv_game_rows excel

'''
# fv_game_rows 요약 정보 추출
export_rows = []
for row in fv_game_rows:
    export_rows.append({
        "date_str": row["date_str"],
        "match_id": row["match_id"],
        "home_team": row["home_team"],
        "away_team": row["away_team"],
        "home_game_date": row["df_home"]["game_date"].values[0] if row["df_home"] is not None else None,
        "away_game_date": row["df_away"]["game_date"].values[0] if row["df_away"] is not None else None,
        "reason": row["reason"]
    })

# DataFrame 변환
export_df = pd.DataFrame(export_rows)

# 엑셀 파일로 저장
excel_path = "fv_game_rows_summary.xlsx"
export_df.to_excel(excel_path, index=False)

print(f"📁 저장 완료: {excel_path}")
'''

📁 저장 완료: fv_game_rows_summary.xlsx


In [11]:
# 기준 열 구성 (set)
set_base_columns = set(df_fv.columns)

# 불일치 여부 추적
home_mismatch_found = False
away_mismatch_found = False

# 검사
for i, row in enumerate(fv_game_rows):
    if row["df_home"] is not None:
        if set(row["df_home"].columns) != set_base_columns:
            print(f"⚠️ 홈 열 이름 불일치 (순서 무시) at {i}: {row['match_id']}")
            home_mismatch_found = True
    if row["df_away"] is not None:
        if set(row["df_away"].columns) != set_base_columns:
            print(f"⚠️ 어웨이 열 이름 불일치 (순서 무시) at {i}: {row['match_id']}")
            away_mismatch_found = True

# 이상 없는 경우 출력
if not home_mismatch_found:
    print("✅ 모든 홈팀 df 열 구성(set 기준)이 일치합니다.")
if not away_mismatch_found:
    print("✅ 모든 어웨이팀 df 열 구성(set 기준)이 일치합니다.")


✅ 모든 홈팀 df 열 구성(set 기준)이 일치합니다.
✅ 모든 어웨이팀 df 열 구성(set 기준)이 일치합니다.


In [12]:
# 유효한 매치만 필터링
fv_game_rows_valid = [
    row for row in fv_game_rows
    if row["df_home"] is not None and row["df_away"] is not None
]

In [13]:
print(type(fv_game_rows_valid))
print(type(fv_game_rows_valid[0]))
print(fv_game_rows_valid[0].keys())  # 딕셔너리 키 구조(열) 보기
print("fv_game_rows_valid 행 개수:", len(fv_game_rows_valid))

home_count = sum(1 for row in fv_game_rows_valid if row["df_home"] is not None)
away_count = sum(1 for row in fv_game_rows_valid if row["df_away"] is not None)

print(f"유효한 홈팀 경기 수: {home_count}")
print(f"유효한 어웨이팀 경기 수: {away_count}")

valid_match_count = sum(1 for row in fv_game_rows_valid if row["df_home"] is not None and row["df_away"] is not None)
print(f"예측 가능한 경기 수 (홈+어웨이 모두 있음): {valid_match_count}")



# 시각화용 정보만 추출
preview_rows = []
for row in fv_game_rows_valid[:5]:  # 앞 5개만 확인
    preview_rows.append({
        "date_str": row["date_str"],
        "match_id": row["match_id"],
        "home_team": row["home_team"],
        "away_team": row["away_team"],
        "home_game_date": row["df_home"]["game_date"].values[0] if row["df_home"] is not None else None,
        "away_game_date": row["df_away"]["game_date"].values[0] if row["df_away"] is not None else None,
        "reason": row["reason"]
    })

# DataFrame으로 변환 및 출력
preview_df = pd.DataFrame(preview_rows)
display(preview_df)

<class 'list'>
<class 'dict'>
dict_keys(['date_str', 'match_id', 'home_team', 'away_team', 'df_home', 'df_away', 'reason'])
fv_game_rows_valid 행 개수: 304
유효한 홈팀 경기 수: 304
유효한 어웨이팀 경기 수: 304
예측 가능한 경기 수 (홈+어웨이 모두 있음): 304


,date_str,match_id,home_team,away_team,home_game_date,away_game_date,reason
0,250602,COL@MIA,MIA,COL,2025-06-01,2025-06-01,None
1,250603,COL@MIA,MIA,COL,2025-06-02,2025-06-02,None
2,250603,DET@CWS,CWS,DET,2025-06-02,2025-06-02,None
3,250603,LAA@BOS,BOS,LAA,2025-06-02,2025-06-02,None
4,250603,MIL@CIN,CIN,MIL,2025-06-02,2025-06-02,None


In [ ]:
# fv_game_rows_valid excel 

'''
# fv_game_rows_valid 요약 정보 추출
export_rows = []
for row in fv_game_rows_valid:
    export_rows.append({
        "date_str": row["date_str"],
        "match_id": row["match_id"],
        "home_team": row["home_team"],
        "away_team": row["away_team"],
        "home_game_date": row["df_home"]["game_date"].values[0] if row["df_home"] is not None else None,
        "away_game_date": row["df_away"]["game_date"].values[0] if row["df_away"] is not None else None,
        "reason": row["reason"]
    })

# DataFrame 변환
export_df = pd.DataFrame(export_rows)

# 엑셀 파일로 저장
excel_path = "fv_game_rows_valid_summary.xlsx"
export_df.to_excel(excel_path, index=False)

print(f"📁 저장 완료: {excel_path}")

'''

📁 저장 완료: fv_game_rows_valid_summary.xlsx


In [15]:
# 기준 열 구성 (set)
set_base_columns = set(df_fv.columns)

# 불일치 여부 추적
home_mismatch_found = False
away_mismatch_found = False

# 검사
for i, row in enumerate(fv_game_rows_valid):
    if row["df_home"] is not None:
        if set(row["df_home"].columns) != set_base_columns:
            print(f"⚠️ 홈 열 이름 불일치 (순서 무시) at {i}: {row['match_id']}")
            home_mismatch_found = True
    if row["df_away"] is not None:
        if set(row["df_away"].columns) != set_base_columns:
            print(f"⚠️ 어웨이 열 이름 불일치 (순서 무시) at {i}: {row['match_id']}")
            away_mismatch_found = True

# 이상 없는 경우 출력
if not home_mismatch_found:
    print("✅ 모든 홈팀 df 열 구성(set 기준)이 일치합니다.")
if not away_mismatch_found:
    print("✅ 모든 어웨이팀 df 열 구성(set 기준)이 일치합니다.")


✅ 모든 홈팀 df 열 구성(set 기준)이 일치합니다.
✅ 모든 어웨이팀 df 열 구성(set 기준)이 일치합니다.


# 2. Preprocessing

## 2-1 drop, fill, X y split

In [16]:
# meta, drop 분리
meta_features = [
    'game_pk',
    'inning', 'inning_topbot',
    'home_team', 'away_team',
    'game_date',
    'batter', 'pitcher',
    'post_home_score', 'post_away_score',  # 최종 점수 → label
    'home_score', 'away_score',
    'events',
    
    # fvdf 파일에만 있는 변수
    'date_str', 'match_id',
]

drop_features = [
    'player_name',
    'description',
    
    # 사용되지 않는 deprecated
    'spin_dir', 'spin_rate_deprecated',
    'break_angle_deprecated', 'break_length_deprecated',
    'tfs_deprecated', 'tfs_zulu_deprecated',
    
    
    'des',
    'umpire',
    'sv_id',
    'pitcher_days_until_next_game', 'batter_days_until_next_game',
    
    # 중복 또는 의미 중복
    'age_pit_legacy', 'age_bat_legacy', 'home_score_diff',

    # 타겟 정보 포함 가능성 (데이터 누수 우려)
    'post_bat_score', 'post_fld_score',
    'delta_home_win_exp', 'delta_run_exp', 'delta_pitcher_run_exp',
    'home_win_exp', 'bat_win_exp',
    'bat_score', 'fld_score',
    
    # 단일 시즌 훈련 시 불필요
    'game_year', 'season',
    
    # fielder
    'fielder_2', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6', 'fielder_7', 'fielder_8', 'fielder_9',
]



print("meta_features len : " + str(len(meta_features)))
print("drop_features len : " + str(len(drop_features)))

meta_features len : 15
drop_features len : 35


In [17]:
print("fv_game_rows_valid 행 개수:", len(fv_game_rows_valid))

fv_game_rows_valid 행 개수: 304


In [ ]:
def preprocess_X_meta(df_prepro, meta_features, drop_features):
    # meta
    df_meta = df_prepro[meta_features].copy()

    # X
    df_X = df_prepro.drop(columns=meta_features + drop_features, errors='ignore').copy()

    return df_X, df_meta


# 전체 매치 처리 (fv_game_rows_valid 기준)
fv_game_rows_processed = []
y_mfv_list = []

for row in fv_game_rows_valid:
    # 홈/어웨이 전처리
    df_home_prepro, df_home_meta = preprocess_X_meta(row["df_home"].copy(), meta_features, drop_features)
    df_away_prepro, df_away_meta = preprocess_X_meta(row["df_away"].copy(), meta_features, drop_features)

    # df_fv에서 해당 매치 레이블 추출
    game_date = pd.to_datetime(row["date_str"], format="%y%m%d")
    cond = (
        (df_fv["game_date"] == game_date) &
        (df_fv["home_team"] == row["home_team"]) &
        (df_fv["away_team"] == row["away_team"])
    )
    df_match = df_fv[cond]

    if df_match.empty:
        continue  # 일치하는 경기 없으면 스킵

    post_home_score = df_match["post_home_score"].max()
    post_away_score = df_match["post_away_score"].max()

    label = (
        1 if post_home_score > post_away_score else
        0 if post_home_score < post_away_score else
        1  # 무승부는 홈 승 처리
    )

    # 저장
    fv_game_rows_processed.append({
        "date_str": row["date_str"],
        "match_id": row["match_id"],
        "home_team": row["home_team"],
        "away_team": row["away_team"],
        "X_fv_a_home": df_home_prepro,
        "X_fv_b_away": df_away_prepro,
        "meta_fv_a_home": df_home_meta,
        "meta_fv_b_away": df_away_meta,
    })

    y_mfv_list.append({
        "date_str": row["date_str"],
        "match_id": row["match_id"],
        
        "home_team": row["home_team"],
        "away_team": row["away_team"],
        
        "post_home_score": post_home_score,
        "post_away_score": post_away_score,
        
        "y": label
    })


# y DataFrame 생성
df_y_mfv = pd.DataFrame(y_mfv_list)


In [19]:
not_matched_rows = []

for row in fv_game_rows_valid:
    game_date = pd.to_datetime(row["date_str"], format="%y%m%d")
    cond = (
        (df_fv["game_date"] == game_date) &
        (df_fv["home_team"] == row["home_team"]) &
        (df_fv["away_team"] == row["away_team"])
    )
    if df_fv[cond].empty:
        not_matched_rows.append(row)

print(f"총 누락된 매치 수: {len(not_matched_rows)}")
for r in not_matched_rows:
    print(r["date_str"], r["match_id"])


KeyboardInterrupt: 

In [ ]:
# fv_game_rows_processed excel
'''

fv_processed_summary = []

for row in fv_game_rows_processed:
    def get_shape_str(df):
        return f"{df.shape[0]}x{df.shape[1]}" if df is not None else None

    fv_processed_summary.append({
        "date_str": row["date_str"],
        "match_id": row["match_id"],
        "home_team": row["home_team"],
        "away_team": row["away_team"],
        "X_fv_a_home": get_shape_str(row.get("X_fv_a_home")),
        "X_fv_b_away": get_shape_str(row.get("X_fv_b_away")),
        "meta_fv_a_home": get_shape_str(row.get("meta_fv_a_home")),
        "meta_fv_b_away": get_shape_str(row.get("meta_fv_b_away")),
    })

df_fv_processed_summary = pd.DataFrame(fv_processed_summary)

# 저장
df_fv_processed_summary.to_excel("fv_game_rows_processed_summary.xlsx", index=False)
df_y_mfv.to_excel("fv_game_rows_processed_summary_y.xlsx", index=False)

'''

'\n\nfv_processed_summary = []\n\nfor row in fv_game_rows_processed:\n    def get_shape_str(df):\n        return f"{df.shape[0]}x{df.shape[1]}" if df is not None else None\n\n    fv_processed_summary.append({\n        "date_str": row["date_str"],\n        "match_id": row["match_id"],\n        "home_team": row["home_team"],\n        "away_team": row["away_team"],\n        "X_fv_a_home": get_shape_str(row.get("X_fv_a_home")),\n        "X_fv_b_away": get_shape_str(row.get("X_fv_b_away")),\n        "meta_fv_a_home": get_shape_str(row.get("meta_fv_a_home")),\n        "meta_fv_b_away": get_shape_str(row.get("meta_fv_b_away")),\n    })\n\ndf_fv_processed_summary = pd.DataFrame(fv_processed_summary)\n\n# 저장\ndf_fv_processed_summary.to_excel("fv_game_rows_processed_summary.xlsx", index=False)\ndf_y_mfv.to_excel("fv_game_rows_processed_summary_y.xlsx", index=False)\n\n'

In [ ]:
# 사실상 drop 되는 feature

# 1) drop by missing
# 이유 : 결측치 50%-80% (drop 여부 결정)
high_missing_features_to_drop = [
    'hit_location',
    'bb_type',
    'on_3b',
    'on_2b',
    'on_1b',
    'hc_x',
    'hc_y',
    'hit_distance_sc',
    'launch_speed',
    'launch_angle',
    'estimated_ba_using_speedangle',
    'estimated_woba_using_speedangle',
    'woba_value',
    'woba_denom',
    'babip_value',
    'iso_value',
    'launch_speed_angle',
    'bat_speed',
    'swing_length',
    'estimated_slg_using_speedangle',
    'hyper_speed',
    'attack_angle',
    'attack_direction',
    'swing_path_tilt',
    'intercept_ball_minus_batter_pos_x_inches',
    'intercept_ball_minus_batter_pos_y_inches'
]


fv_game_rows_processed_dropped = []

for row in fv_game_rows_processed:
    # X 데이터에 drop 적용
    X_home = row["X_fv_a_home"].drop(columns=high_missing_features_to_drop, errors="ignore")
    X_away = row["X_fv_b_away"].drop(columns=high_missing_features_to_drop, errors="ignore")

    # 저장
    fv_game_rows_processed_dropped.append({
        "date_str": row["date_str"],
        "match_id": row["match_id"],
        "home_team": row["home_team"],
        "away_team": row["away_team"],
        "X_fv_a_home": X_home,
        "X_fv_b_away": X_away,
        "meta_fv_a_home": row["meta_fv_a_home"],
        "meta_fv_b_away": row["meta_fv_b_away"],
    })


In [ ]:
# X fill

# --------------- 컬럼 정의
# 평균으로 채울 컬럼
fill_mean_cols = [
    'release_speed', 'release_pos_x', 'release_pos_z', 'pfx_x', 'pfx_z',
    'plate_x', 'plate_z', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az',
    'sz_top', 'sz_bot', 'effective_speed', 'release_spin_rate',
    'release_extension', 'release_pos_y', 'spin_axis', 'api_break_z_with_gravity',
    'api_break_x_arm', 'api_break_x_batter_in', 'arm_angle'
]

# 최빈값으로 채울 컬럼
fill_mode_cols = ['zone']

# 'Unknown'으로 채울 object 타입 컬럼
fill_unknown_cols = ['pitch_type', 'pitch_name']

# 고정값으로 채울 컬럼
fill_constant = {
    'if_fielding_alignment': 'Standard',
    'of_fielding_alignment': 'Standard',
    'delta_run_exp': 0,
    'delta_pitcher_run_exp': 0,
    'pitcher_days_since_prev_game': 0,
    'batter_days_since_prev_game': 0
}


# --------------- 적용

fv_game_rows_filled = []

for row in fv_game_rows_processed_dropped:
    X_home = row["X_fv_a_home"].copy()
    X_away = row["X_fv_b_away"].copy()

    # 1. 평균으로 채움
    for col in fill_mean_cols:
        for df in [X_home, X_away]:
            if col in df.columns:
                mean_val = df[col].mean()
                if pd.api.types.is_integer_dtype(df[col].dtype):
                    mean_val = int(round(mean_val))
                df[col].fillna(mean_val, inplace=True)

    # 2. 최빈값으로 채움
    for col in fill_mode_cols:
        for df in [X_home, X_away]:
            if col in df.columns and not df[col].mode().empty:
                mode_val = df[col].mode().iloc[0]
                df[col].fillna(mode_val, inplace=True)

    # 3. Unknown 채움
    for col in fill_unknown_cols:
        for df in [X_home, X_away]:
            if col in df.columns:
                df[col].fillna("Unknown", inplace=True)

    # 4. 고정값 채움
    for col, val in fill_constant.items():
        for df in [X_home, X_away]:
            if col in df.columns:
                df[col].fillna(val, inplace=True)

    # 저장
    fv_game_rows_filled.append({
        "date_str": row["date_str"],
        "match_id": row["match_id"],
        "home_team": row["home_team"],
        "away_team": row["away_team"],
        "X_fv_a_home": X_home,
        "X_fv_b_away": X_away,
        "meta_fv_a_home": row["meta_fv_a_home"],
        "meta_fv_b_away": row["meta_fv_b_away"]
    })


In [ ]:

# 전체 결측 컬럼 추적용
missing_info = []

for row in fv_game_rows_filled:
    for team_type in ["X_fv_a_home", "X_fv_b_away"]:
        df = row[team_type]
        total_missing = df.isnull().sum().sum()

        if total_missing > 0:
            missing_by_col = df.isnull().sum()
            missing_cols = missing_by_col[missing_by_col > 0]

            missing_info.append({
                "date_str": row["date_str"],
                "match_id": row["match_id"],
                "team_type": team_type,
                "total_missing": total_missing,
                "missing_columns": missing_cols
            })

# 결과 출력
if not missing_info:
    print("✅ 모든 경기 데이터에서 결측값이 없습니다.")
else:
    print(f"⚠️ 총 {len(missing_info)}개의 경기 데이터에서 결측값이 발견되었습니다.\n")

    for info in missing_info:
        print(f"📅 {info['date_str']} | {info['match_id']} | {info['team_type']} — 총 결측값: {info['total_missing']}")
        display(info["missing_columns"])
        print("-" * 60)

✅ 모든 경기 데이터에서 결측값이 없습니다.


## 2-2 Encoding

In [ ]:

# 1. Label Encoding
label_encode_cols = ['pitch_type', 'type', 'zone', 'pitch_name']
ordinal_enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

# fit 기준: 전체 home+away 데이터를 합쳐서 학습
X_concat_for_fit = pd.concat(
    [row["X_fv_a_home"][label_encode_cols] for row in fv_game_rows_filled] +
    [row["X_fv_b_away"][label_encode_cols] for row in fv_game_rows_filled],
    axis=0
)
ordinal_enc.fit(X_concat_for_fit)

# 2. One-Hot Encoding 대상 컬럼과 전체 가능한 카테고리 확보
one_hot_cols = ['game_type', 'stand', 'p_throws', 'if_fielding_alignment', 'of_fielding_alignment']

# 전체 가능한 dummy 컬럼 추출 (빈 DataFrame에 대해 한 번 get_dummies 수행)
X_concat_all = pd.concat(
    [row["X_fv_a_home"] for row in fv_game_rows_filled] +
    [row["X_fv_b_away"] for row in fv_game_rows_filled],
    axis=0
)
dummy_all_columns = pd.get_dummies(X_concat_all[one_hot_cols], columns=one_hot_cols).columns.tolist()

# 3. Drop
fielder_cols = [f'fielder_{i}' for i in range(2, 10)]
drop_cols = fielder_cols + [
    'age_pit_legacy', 'age_bat_legacy', 'home_score_diff',
    'post_bat_score', 'post_fld_score',
    'delta_home_win_exp', 'delta_run_exp', 'delta_pitcher_run_exp',
    'home_win_exp', 'bat_win_exp',
    'bat_score', 'fld_score',
    'game_year', 'season',
]

# 전처리 결과 저장
fv_game_rows_encoded = []

for row in fv_game_rows_filled:
    processed_row = {
        "date_str": row["date_str"],
        "match_id": row["match_id"],
        "home_team": row["home_team"],
        "away_team": row["away_team"],
        "meta_fv_a_home": row["meta_fv_a_home"],
        "meta_fv_b_away": row["meta_fv_b_away"],
    }

    for team_key in ["X_fv_a_home", "X_fv_b_away"]:
        df_encoded = row[team_key].copy()

        # 1. Ordinal Encoding
        df_encoded[label_encode_cols] = ordinal_enc.transform(df_encoded[label_encode_cols])

        # 2. One-Hot Encoding
        df_dummies = pd.get_dummies(df_encoded[one_hot_cols], columns=one_hot_cols)

        # 누락된 컬럼 보완
        for col in dummy_all_columns:
            if col not in df_dummies.columns:
                df_dummies[col] = 0
        df_dummies = df_dummies[dummy_all_columns]  # 컬럼 순서 맞춤

        # 기존 원본에서 drop하고, one-hot 덧붙임
        df_encoded.drop(columns=one_hot_cols + drop_cols, errors='ignore', inplace=True)
        df_encoded = pd.concat([df_encoded, df_dummies], axis=1)

        processed_row[team_key] = df_encoded

    fv_game_rows_encoded.append(processed_row)

# 희귀 one-hot 컬럼 제거
rare_alignment_cols = [
    'if_fielding_alignment_Infield shade',
    'of_fielding_alignment_4th outfielder'
]

for row in fv_game_rows_encoded:
    for team_key in ["X_fv_a_home", "X_fv_b_away"]:
        row[team_key].drop(columns=rare_alignment_cols, errors='ignore', inplace=True)


In [ ]:
# 대표 df 선택 (예: 첫 번째 row의 X_fv_a_home)
sample_df = fv_game_rows_encoded[0]["X_fv_a_home"]

# 전체 컬럼 리스트
columns = sample_df.columns.tolist()

# 전체 컬럼 개수 출력
print(f"총 컬럼 수: {len(columns)}\n")

# 5개씩 한 줄로 출력
for i in range(0, len(columns), 5):
    print(columns[i:i+5])


총 컬럼 수: 49

['pitch_type', 'release_speed', 'release_pos_x', 'release_pos_z', 'zone']
['type', 'balls', 'strikes', 'pfx_x', 'pfx_z']
['plate_x', 'plate_z', 'outs_when_up', 'vx0', 'vy0']
['vz0', 'ax', 'ay', 'az', 'sz_top']
['sz_bot', 'effective_speed', 'release_spin_rate', 'release_extension', 'release_pos_y']
['at_bat_number', 'pitch_number', 'pitch_name', 'spin_axis', 'bat_score_diff']
['age_pit', 'age_bat', 'n_thruorder_pitcher', 'n_priorpa_thisgame_player_at_bat', 'pitcher_days_since_prev_game']
['batter_days_since_prev_game', 'api_break_z_with_gravity', 'api_break_x_arm', 'api_break_x_batter_in', 'arm_angle']
['game_type_R', 'stand_L', 'stand_R', 'p_throws_L', 'p_throws_R']
['if_fielding_alignment_Standard', 'if_fielding_alignment_Strategic', 'of_fielding_alignment_Standard', 'of_fielding_alignment_Strategic']


In [ ]:
# fv_game_rows_encoded -> excel


'''
fv_encoded_summary = []

def get_shape(df):
    return f"{df.shape[0]}x{df.shape[1]}" if df is not None else "None"

for row in fv_game_rows_encoded:
    fv_encoded_summary.append({
        "date_str": row["date_str"],
        "match_id": row["match_id"],
        "home_team": row["home_team"],
        "away_team": row["away_team"],
        "X_fv_a_home": get_shape(row["X_fv_a_home"]),
        "X_fv_b_away": get_shape(row["X_fv_b_away"]),
        "meta_fv_a_home": get_shape(row["meta_fv_a_home"]),
        "meta_fv_b_away": get_shape(row["meta_fv_b_away"]),
    })

# DataFrame 생성
df_encoded_summary = pd.DataFrame(fv_encoded_summary)

# 저장
df_encoded_summary.to_excel("fv_game_rows_encoded_summary.xlsx", index=False)
print("✅ 요약 엑셀 파일 저장 완료: fv_game_rows_encoded_summary.xlsx")

'''

'\nfv_encoded_summary = []\n\ndef get_shape(df):\n    return f"{df.shape[0]}x{df.shape[1]}" if df is not None else "None"\n\nfor row in fv_game_rows_encoded:\n    fv_encoded_summary.append({\n        "date_str": row["date_str"],\n        "match_id": row["match_id"],\n        "home_team": row["home_team"],\n        "away_team": row["away_team"],\n        "X_fv_a_home": get_shape(row["X_fv_a_home"]),\n        "X_fv_b_away": get_shape(row["X_fv_b_away"]),\n        "meta_fv_a_home": get_shape(row["meta_fv_a_home"]),\n        "meta_fv_b_away": get_shape(row["meta_fv_b_away"]),\n    })\n\n# DataFrame 생성\ndf_encoded_summary = pd.DataFrame(fv_encoded_summary)\n\n# 저장\ndf_encoded_summary.to_excel("fv_game_rows_encoded_summary.xlsx", index=False)\nprint("✅ 요약 엑셀 파일 저장 완료: fv_game_rows_encoded_summary.xlsx")\n\n'

In [ ]:
# X_fv_a_home, X_fv_b_away 의 모든 df들 col 동일 상태인지 확인

# 기준 컬럼: 첫 번째 row의 컬럼 목록
ref_cols_a = fv_game_rows_encoded[0]["X_fv_a_home"].columns.tolist()
ref_cols_b = fv_game_rows_encoded[0]["X_fv_b_away"].columns.tolist()

# 불일치 추적용 리스트
mismatch_rows = []

for idx, row in enumerate(fv_game_rows_encoded):
    cols_a = row["X_fv_a_home"].columns.tolist()
    cols_b = row["X_fv_b_away"].columns.tolist()

    if cols_a != ref_cols_a or cols_b != ref_cols_b:
        mismatch_rows.append({
            "index": idx,
            "match_id": row["match_id"],
            "home_team": row["home_team"],
            "away_team": row["away_team"],
            "home_cols_mismatch": cols_a != ref_cols_a,
            "away_cols_mismatch": cols_b != ref_cols_b,
            "num_home_cols": len(cols_a),
            "num_away_cols": len(cols_b),
        })

# 결과 출력
if mismatch_rows:
    import pandas as pd
    df_mismatches = pd.DataFrame(mismatch_rows)
    print("❌ 컬럼 수 또는 순서가 일치하지 않는 데이터가 있습니다:")
    display(df_mismatches)
else:
    print("✅ 모든 X_fv_a_home / X_fv_b_away 컬럼이 동일합니다.")


✅ 모든 X_fv_a_home / X_fv_b_away 컬럼이 동일합니다.


## 2-3 Outlier

## 2-4 Feature Engineering

In [ ]:
# 2-4. Time-Aware Feature Engineering

def create_time_aware_feature(df_encoded):
    def trend_calc(x):
        if len(x) < 2:
            return np.nan
        return np.polyfit(range(len(x)), x, 1)[0]

    def generate_time_features(base_series, prefix):
        return {
            f"{prefix}_rolling_avg_5": base_series.rolling(window=5, min_periods=1).mean(),
            f"{prefix}_lag_1": base_series.shift(1),
            f"{prefix}_diff_1": base_series.diff(),
        }

    all_features = []

    spin_eff = df_encoded["release_spin_rate"] / df_encoded["release_speed"]
    all_features.append(pd.DataFrame(generate_time_features(spin_eff, "spin_efficiency"), index=df_encoded.index))

    speed_gap = df_encoded["release_speed"] - df_encoded["effective_speed"]
    all_features.append(pd.DataFrame(generate_time_features(speed_gap, "speed_gap"), index=df_encoded.index))

    release_pos_angle = np.arctan(df_encoded["release_pos_z"] / (df_encoded["release_pos_x"] + 1e-6))
    all_features.append(pd.DataFrame(generate_time_features(release_pos_angle, "release_pos_angle"), index=df_encoded.index))

    plate_distance = np.sqrt(df_encoded["plate_x"]**2 + df_encoded["plate_z"]**2)
    all_features.append(pd.DataFrame(generate_time_features(plate_distance, "plate_distance"), index=df_encoded.index))

    init_velocity_vector = np.sqrt(df_encoded["vx0"]**2 + df_encoded["vy0"]**2 + df_encoded["vz0"]**2)
    all_features.append(pd.DataFrame(generate_time_features(init_velocity_vector, "init_velocity_vector"), index=df_encoded.index))

    pitch_movement_vector = np.sqrt(df_encoded["pfx_x"]**2 + df_encoded["pfx_z"]**2)
    all_features.append(pd.DataFrame(generate_time_features(pitch_movement_vector, "pitch_movement_vector"), index=df_encoded.index))

    speed_per_extension = df_encoded["release_speed"] / (df_encoded["release_extension"] + 1e-6)
    all_features.append(pd.DataFrame(generate_time_features(speed_per_extension, "speed_per_extension"), index=df_encoded.index))

    release_mechanics_profile = df_encoded["release_pos_y"] + df_encoded["release_extension"]
    all_features.append(pd.DataFrame(generate_time_features(release_mechanics_profile, "release_mechanics_profile"), index=df_encoded.index))

    vertical_spin_effect = df_encoded["spin_axis"] * df_encoded["pfx_z"]
    all_features.append(pd.DataFrame(generate_time_features(vertical_spin_effect, "vertical_spin_effect"), index=df_encoded.index))

    horizontal_spin_effect = df_encoded["release_spin_rate"] / (df_encoded["pfx_x"] + 1e-6)
    all_features.append(pd.DataFrame(generate_time_features(horizontal_spin_effect, "horizontal_spin_effect"), index=df_encoded.index))

    df_taf = pd.concat(all_features, axis=1)
    return df_taf

# 적용 및 저장
fv_game_rows_taf = []

for row in fv_game_rows_encoded:
    taf_row = {
        "date_str": row["date_str"],
        "match_id": row["match_id"],
        "home_team": row["home_team"],
        "away_team": row["away_team"],
        "meta_fv_a_home": row["meta_fv_a_home"],
        "meta_fv_b_away": row["meta_fv_b_away"],
        "X_fv_a_home_taf": create_time_aware_feature(row["X_fv_a_home"]),
        "X_fv_b_away_taf": create_time_aware_feature(row["X_fv_b_away"]),
    }

    fv_game_rows_taf.append(taf_row)


In [ ]:
# taf_shape_summary excel


taf_shape_summary = []

for row in fv_game_rows_taf:
    taf_shape_summary.append({
        "date_str": row["date_str"],
        "match_id": row["match_id"],
        "home_team": row["home_team"],
        "away_team": row["away_team"],
        "X_fv_a_home_taf_shape": f"{row['X_fv_a_home_taf'].shape[0]}x{row['X_fv_a_home_taf'].shape[1]}",
        "X_fv_b_away_taf_shape": f"{row['X_fv_b_away_taf'].shape[0]}x{row['X_fv_b_away_taf'].shape[1]}",
        "meta_fv_a_home_shape": f"{row['meta_fv_a_home'].shape[0]}x{row['meta_fv_a_home'].shape[1]}",
        "meta_fv_b_away_shape": f"{row['meta_fv_b_away'].shape[0]}x{row['meta_fv_b_away'].shape[1]}",
    })

# DataFrame 변환
df_shape_summary = pd.DataFrame(taf_shape_summary)

# Excel 저장
df_shape_summary.to_excel("taf_feature_shape_summary.xlsx", index=False)
print("✅ Saved to taf_feature_shape_summary.xlsx")


✅ Saved to taf_feature_shape_summary.xlsx


In [ ]:
# 대표 df 선택 (예: 첫 번째 row의 X_fv_a_home_taf)
sample_taf_df = fv_game_rows_taf[0]["X_fv_a_home_taf"]

# 컬럼 목록 추출
taf_cols = list(sample_taf_df.columns)

# 출력
print(f"🧩 총 {len(taf_cols)}개의 TAF 컬럼:\n")

for i in range(0, len(taf_cols), 3):
    print(', '.join(taf_cols[i:i+3]))


🧩 총 30개의 TAF 컬럼:

spin_efficiency_rolling_avg_5, spin_efficiency_lag_1, spin_efficiency_diff_1
speed_gap_rolling_avg_5, speed_gap_lag_1, speed_gap_diff_1
release_pos_angle_rolling_avg_5, release_pos_angle_lag_1, release_pos_angle_diff_1
plate_distance_rolling_avg_5, plate_distance_lag_1, plate_distance_diff_1
init_velocity_vector_rolling_avg_5, init_velocity_vector_lag_1, init_velocity_vector_diff_1
pitch_movement_vector_rolling_avg_5, pitch_movement_vector_lag_1, pitch_movement_vector_diff_1
speed_per_extension_rolling_avg_5, speed_per_extension_lag_1, speed_per_extension_diff_1
release_mechanics_profile_rolling_avg_5, release_mechanics_profile_lag_1, release_mechanics_profile_diff_1
vertical_spin_effect_rolling_avg_5, vertical_spin_effect_lag_1, vertical_spin_effect_diff_1
horizontal_spin_effect_rolling_avg_5, horizontal_spin_effect_lag_1, horizontal_spin_effect_diff_1


In [ ]:
# FVDF 기반 TAF 결측값 확인

for row in fv_game_rows_taf:
    for key in ["X_fv_a_home_taf", "X_fv_b_away_taf"]:
        df = row[key]
        missing_cols = df.columns[df.isnull().any()]
        if len(missing_cols) > 0:
            print(f"🔍 {row['match_id']} ({key}) has missing values in:")
            print(df[missing_cols].isnull().sum())
            print("-" * 40)
        else:
            print(f"✅ {row['match_id']} ({key}) has no missing values.")


🔍 COL@MIA (X_fv_a_home_taf) has missing values in:
spin_efficiency_lag_1               1
spin_efficiency_diff_1              1
speed_gap_lag_1                     1
speed_gap_diff_1                    1
release_pos_angle_lag_1             1
release_pos_angle_diff_1            1
plate_distance_lag_1                1
plate_distance_diff_1               1
init_velocity_vector_lag_1          1
init_velocity_vector_diff_1         1
pitch_movement_vector_lag_1         1
pitch_movement_vector_diff_1        1
speed_per_extension_lag_1           1
speed_per_extension_diff_1          1
release_mechanics_profile_lag_1     1
release_mechanics_profile_diff_1    1
vertical_spin_effect_lag_1          1
vertical_spin_effect_diff_1         1
horizontal_spin_effect_lag_1        1
horizontal_spin_effect_diff_1       1
dtype: int64
----------------------------------------
🔍 COL@MIA (X_fv_b_away_taf) has missing values in:
spin_efficiency_lag_1               1
spin_efficiency_diff_1              1
speed_ga

In [ ]:
# fv_game_rows_taf의 TAF 결측값을 bfill로 채우기 (lag, diff 처리 목적)

for row in fv_game_rows_taf:
    for key in ["X_fv_a_home_taf", "X_fv_b_away_taf"]:
        df = row[key]
        df.fillna(method="bfill", inplace=True)


In [ ]:
# FVDF 기반 TAF 결측값 전체 검사 (요약 버전)
has_missing = False

for row in fv_game_rows_taf:
    for key in ["X_fv_a_home_taf", "X_fv_b_away_taf"]:
        df = row[key]
        if df.isnull().values.any():
            has_missing = True
            break
    if has_missing:
        break

if has_missing:
    print("❗ FVDF TAF 중 하나 이상의 경기에서 결측값이 발견되었습니다.")
else:
    print("✅ 모든 FVDF TAF 데이터프레임에 결측값이 없습니다.")


✅ 모든 FVDF TAF 데이터프레임에 결측값이 없습니다.


In [ ]:
# 선택할 TAF 컬럼
selected_taf_cols = [
    'spin_efficiency_rolling_avg_5',
    'speed_gap_diff_1',
    'pitch_movement_vector_rolling_avg_5',
    'vertical_spin_effect_lag_1'
]

fv_game_rows_fe = []

for enc_row, taf_row in zip(fv_game_rows_encoded, fv_game_rows_taf):
    # 기본 메타 복사
    fe_row = {
        "date_str": enc_row["date_str"],
        "match_id": enc_row["match_id"],
        "home_team": enc_row["home_team"],
        "away_team": enc_row["away_team"],
        "meta_fv_a_home": enc_row["meta_fv_a_home"],
        "meta_fv_b_away": enc_row["meta_fv_b_away"],
    }

    # 팀별 feature 병합
    for team_key in ["X_fv_a_home", "X_fv_b_away"]:
        X_encoded = enc_row[team_key].copy()
        X_taf = taf_row[team_key + "_taf"][selected_taf_cols].copy()

        # 병합
        X_fe = pd.concat([X_encoded.reset_index(drop=True), X_taf.reset_index(drop=True)], axis=1)

        fe_row[team_key + "_FE"] = X_fe

    fv_game_rows_fe.append(fe_row)


In [ ]:
# 예시: 첫 번째 row의 X_fv_a_home_FE 선택
sample_df = fv_game_rows_fe[0]["X_fv_a_home_FE"]

# 전체 컬럼 리스트
columns = sample_df.columns.tolist()

# 총 개수 출력
print(f"📊 Total {len(columns)} columns in X_fv_a_home_FE\n")

# 5개씩 나눠서 출력
for i in range(0, len(columns), 5):
    print(columns[i:i+5])


📊 Total 53 columns in X_fv_a_home_FE

['pitch_type', 'release_speed', 'release_pos_x', 'release_pos_z', 'zone']
['type', 'balls', 'strikes', 'pfx_x', 'pfx_z']
['plate_x', 'plate_z', 'outs_when_up', 'vx0', 'vy0']
['vz0', 'ax', 'ay', 'az', 'sz_top']
['sz_bot', 'effective_speed', 'release_spin_rate', 'release_extension', 'release_pos_y']
['at_bat_number', 'pitch_number', 'pitch_name', 'spin_axis', 'bat_score_diff']
['age_pit', 'age_bat', 'n_thruorder_pitcher', 'n_priorpa_thisgame_player_at_bat', 'pitcher_days_since_prev_game']
['batter_days_since_prev_game', 'api_break_z_with_gravity', 'api_break_x_arm', 'api_break_x_batter_in', 'arm_angle']
['game_type_R', 'stand_L', 'stand_R', 'p_throws_L', 'p_throws_R']
['if_fielding_alignment_Standard', 'if_fielding_alignment_Strategic', 'of_fielding_alignment_Standard', 'of_fielding_alignment_Strategic', 'spin_efficiency_rolling_avg_5']
['speed_gap_diff_1', 'pitch_movement_vector_rolling_avg_5', 'vertical_spin_effect_lag_1']


In [ ]:
# 결측치 검사
has_missing = False

for i, row in enumerate(fv_game_rows_fe):
    for team_key in ["X_fv_a_home_FE", "X_fv_b_away_FE"]:
        df = row[team_key]
        null_cols = df.columns[df.isnull().any()].tolist()
        if null_cols:
            has_missing = True
            print(f"❗ 결측치 발견 in match_id={row['match_id']} [{team_key}]: {null_cols}")

if not has_missing:
    print("✅ 모든 FE DataFrame에서 결측치 없음")


✅ 모든 FE DataFrame에서 결측치 없음


In [ ]:
# fv_game_rows_fe excel
'''
shape_summary = []

for row in fv_game_rows_fe:
    shape_summary.append({
        "date_str": row["date_str"],
        "match_id": row["match_id"],
        "home_team": row["home_team"],
        "away_team": row["away_team"],
        "X_fv_a_home_FE": f"{row['X_fv_a_home_FE'].shape[0]}x{row['X_fv_a_home_FE'].shape[1]}",
        "X_fv_b_away_FE": f"{row['X_fv_b_away_FE'].shape[0]}x{row['X_fv_b_away_FE'].shape[1]}"
    })

# DataFrame 생성
df_shape_summary = pd.DataFrame(shape_summary)


# Excel로 저장
df_shape_summary.to_excel("fv_game_rows_fe_shape.xlsx", index=False)
'''

'\nshape_summary = []\n\nfor row in fv_game_rows_fe:\n    shape_summary.append({\n        "date_str": row["date_str"],\n        "match_id": row["match_id"],\n        "home_team": row["home_team"],\n        "away_team": row["away_team"],\n        "X_fv_a_home_FE": f"{row[\'X_fv_a_home_FE\'].shape[0]}x{row[\'X_fv_a_home_FE\'].shape[1]}",\n        "X_fv_b_away_FE": f"{row[\'X_fv_b_away_FE\'].shape[0]}x{row[\'X_fv_b_away_FE\'].shape[1]}"\n    })\n\n# DataFrame 생성\ndf_shape_summary = pd.DataFrame(shape_summary)\n\n\n# Excel로 저장\ndf_shape_summary.to_excel("fv_game_rows_fe_shape.xlsx", index=False)\n'

## 2-5 Feature Selection

In [ ]:
'''
X_train_FS = X_train_FE.copy()
X_valid_FS = X_valid_FE.copy()
X_test_FS = X_test_FE.copy()
X_fv_a_home_FS = X_fv_a_home_FE.copy()
X_fv_b_away_FS = X_fv_b_away_FE.copy()
'''

'\nX_train_FS = X_train_FE.copy()\nX_valid_FS = X_valid_FE.copy()\nX_test_FS = X_test_FE.copy()\nX_fv_a_home_FS = X_fv_a_home_FE.copy()\nX_fv_b_away_FS = X_fv_b_away_FE.copy()\n'

## 2-6 Scailing

In [ ]:
'''
X_train_scaling = X_train_FS.copy()
X_valid_scaling = X_valid_FS.copy()
X_test_scaling = X_test_FS.copy()
X_fv_a_home_scaling = X_fv_a_home_FS.copy()
X_fv_b_away_scaling = X_fv_b_away_FS.copy()
'''

'\nX_train_scaling = X_train_FS.copy()\nX_valid_scaling = X_valid_FS.copy()\nX_test_scaling = X_test_FS.copy()\nX_fv_a_home_scaling = X_fv_a_home_FS.copy()\nX_fv_b_away_scaling = X_fv_b_away_FS.copy()\n'

## 2-7 Filtering

In [ ]:
# mean, max, mode list

# 평균값(mean)을 대표값으로 삼는 연속형 수치 변수들
mean_cols = [
    # 투구 및 릴리스 관련 물리 변수들 – 평균 속도나 위치가 경기 스타일을 나타냄
    'release_speed', 'release_pos_x', 'release_pos_z', 'pfx_x', 'pfx_z',
    'plate_x', 'plate_z', 'vx0', 'vy0', 'vz0',
    'ax', 'ay', 'az', 'sz_top', 'sz_bot',
    'effective_speed', 'release_spin_rate', 'release_extension', 'release_pos_y', 'spin_axis',

    # 점수 관련 – 평균 점수 차가 경기 흐름을 반영
    'bat_score_diff',

    # 브레이킹/구질 관련 파생 변수 – 평균 곡선 형태를 반영
    'api_break_z_with_gravity', 'api_break_x_arm', 'api_break_x_batter_in', 'arm_angle',

    # 선수 나이 및 이전 경기 이력 – 평균으로 부드럽게 반영
    'age_pit', 'age_bat', 'pitcher_days_since_prev_game', 'batter_days_since_prev_game',

    # 범주형 인코딩 결과 – 전체 평균으로 경기 경향성 반영
    'game_type_R', 'stand_L', 'stand_R', 'p_throws_L', 'p_throws_R',
    'if_fielding_alignment_Standard', 'if_fielding_alignment_Strategic',
    'of_fielding_alignment_Standard', 'of_fielding_alignment_Strategic'
]


# 최대값(max)을 대표값으로 삼는 변수들
max_cols = [
    # at-bat 또는 투구 순서 – 경기 내에서 가장 깊이 갔던 상황을 반영
    'at_bat_number', 'pitch_number',

    # 투수/타자의 순서나 반복 상황 – 최대 반복 정도를 경기 요약으로 사용
    'n_thruorder_pitcher', 'n_priorpa_thisgame_player_at_bat'
]


# 최빈값(mode)을 대표값으로 삼는 범주형 변수들
mode_cols = [
    # 구종, 스트라이크존, 판정 등 – 가장 자주 등장한 항목이 특징을 잘 나타냄
    'pitch_type', 'zone', 'type', 'balls', 'strikes',
    'outs_when_up', 'pitch_name'
]

# 안전한 mode 함수
def safe_mode(series):
    result = mode(series, nan_policy='omit')
    mode_val = getattr(result, "mode", None)
    if hasattr(mode_val, '__len__') and len(mode_val) > 0:
        return mode_val[0]
    elif np.isscalar(mode_val):
        return mode_val
    else:
        return np.nan
    
    # 기존 목록을 set으로 바꿔 중복 방지
mean_cols_set = set(mean_cols)
max_cols_set = set(max_cols)
mode_cols_set = set(mode_cols)

# 추적용 리스트
new_mean, new_max, new_mode = [], [], []

for col in selected_taf_cols:
    if "rolling_avg" in col:
        if col not in mean_cols_set:
            mean_cols_set.add(col)
            new_mean.append(col)
    elif "lag" in col:
        if col not in max_cols_set:
            max_cols_set.add(col)
            new_max.append(col)
    elif "diff" in col:
        if col not in mode_cols_set:
            mode_cols_set.add(col)
            new_mode.append(col)

# 리스트로 다시 변환
mean_cols = list(mean_cols_set)
max_cols = list(max_cols_set)
mode_cols = list(mode_cols_set)

orig_mean_count = len(mean_cols_set) - len(new_mean)
orig_max_count = len(max_cols_set) - len(new_max)
orig_mode_count = len(mode_cols_set) - len(new_mode)

# 출력
print(f"✅ {len(new_mean)} TAF features added to mean_cols:", new_mean)
print(f"✅ {len(new_max)} TAF features added to max_cols:", new_max)
print(f"✅ {len(new_mode)} TAF features added to mode_cols:", new_mode)

print("\n🔢 최종 feature 수 요약")
print(f"mean_cols: {orig_mean_count} → {len(mean_cols)}")
print(f"max_cols: {orig_max_count} → {len(max_cols)}")
print(f"mode_cols: {orig_mode_count} → {len(mode_cols)}")


✅ 2 TAF features added to mean_cols: ['spin_efficiency_rolling_avg_5', 'pitch_movement_vector_rolling_avg_5']
✅ 1 TAF features added to max_cols: ['vertical_spin_effect_lag_1']
✅ 1 TAF features added to mode_cols: ['speed_gap_diff_1']

🔢 최종 feature 수 요약
mean_cols: 38 → 40
max_cols: 4 → 5
mode_cols: 7 → 8


In [ ]:
# mean, max mode list apply function
def summarize_features(group):
    summary = {}

    for col in mean_cols:
        summary[col] = group[col].mean()
    for col in max_cols:
        summary[col] = group[col].max()
    for col in mode_cols:
        summary[col] = safe_mode(group[col])

    return pd.Series(summary)

In [ ]:
def filter_single_match(X, meta):
    # 0. inning_topbot 붙이기
    X = X.copy()
    X["inning_topbot"] = meta["inning_topbot"].values

    # 1. inning_topbot 기준 groupby
    if not hasattr(filter_single_match, "called"):
        grouped = X.groupby('inning_topbot', group_keys=False).apply(summarize_features).reset_index()
        print("📏 First grouped shape:", grouped.shape)
        display(grouped)
        filter_single_match.called = True
    else:
        grouped = X.groupby('inning_topbot', group_keys=False).apply(summarize_features).reset_index()

    # 2. 정렬
    grouped['inning_topbot'] = pd.Categorical(grouped['inning_topbot'], categories=['Top', 'Bot'], ordered=True)
    grouped = grouped.sort_values('inning_topbot').reset_index(drop=True)

    # 3. pivot
    pivoted = grouped.pivot(columns='inning_topbot')
    pivoted.columns = [f"{col}_{tb.lower()}" for col, tb in pivoted.columns]

    return pivoted




fv_game_rows_filtered = []

for row in fv_game_rows_fe:
    # home
    X_a = row["X_fv_a_home_FE"]
    X_a_filtered = filter_single_match(X_a, row["meta_fv_a_home"])

    # away
    X_b = row["X_fv_b_away_FE"]
    X_b_filtered = filter_single_match(X_b, row["meta_fv_b_away"])

    # 저장
    fv_game_rows_filtered.append({
        "date_str": row["date_str"],
        "match_id": row["match_id"],
        "home_team": row["home_team"],
        "away_team": row["away_team"],
        "meta": row["meta_fv_a_home"],
        "X_fv_home": X_a_filtered,
        "X_fv_away": X_b_filtered,
    })



📏 First grouped shape: (2, 54)


,inning_topbot,of_fielding_alignment_Strategic,spin_efficiency_rolling_avg_5,game_type_R,sz_bot,p_throws_R,spin_axis,api_break_x_arm,p_throws_L,of_fielding_alignment_Standard,...,n_thruorder_pitcher,vertical_spin_effect_lag_1,type,pitch_name,outs_when_up,balls,strikes,pitch_type,zone,speed_gap_diff_1
0,Bot,0.271084,25.791348,1.0,1.532952,0.891566,178.993976,0.322892,0.108434,0.728916,...,3.0,369.25,0.0,11.0,0.0,0.0,0.0,13.0,12.0,0.0
1,Top,0.007299,25.555362,1.0,1.551606,0.197080,176.335766,0.517737,0.802920,0.992701,...,3.0,360.18,1.0,0.0,1.0,0.0,0.0,6.0,11.0,0.0


In [ ]:
# 예시: 첫 번째 row의 feature vector 컬럼 확인
sample_row = fv_game_rows_filtered[0]
columns = sample_row["X_fv"].index.tolist()

print(f"🔍 대표 벡터의 총 {len(columns)}개 컬럼:")
for i in range(0, len(columns), 5):
    print(columns[i:i+5])


KeyError: 'X_fv'

In [ ]:
# fv_game_rows_filtered excel
'''
shape_summary = []

for row in fv_game_rows_filtered:
    shape_summary.append({
        "date_str": row["date_str"],
        "match_id": row["match_id"],
        "home_team": row["home_team"],
        "away_team": row["away_team"],
        "X_fv_len": len(row['X_fv']),  # 또는 row['X_fv'].shape[0]
    })

# DataFrame 생성
df_shape_summary = pd.DataFrame(shape_summary)

# Excel로 저장
df_shape_summary.to_excel("fv_game_rows_filtered_shape.xlsx", index=False)

'''

# 5. Prediction

In [ ]:
pitch_features = [
    'release_speed',  # 투구 속도 (투수 릴리스 시점)
    'release_pos_x',  # 투수 릴리스 위치 (좌우)
    'release_pos_z',  # 투수 릴리스 위치 (높이)
    'release_pos_y',  # 투수 릴리스 위치 (홈과의 거리)
    'pfx_x',  # 투구의 좌우 변화량
    'pfx_z',  # 투구의 상하 변화량
    'vx0',  # 투구 초기 속도 x축 성분
    'vy0',  # 투구 초기 속도 y축 성분
    'vz0',  # 투구 초기 속도 z축 성분
    'ax',  # 투구 가속도 x축
    'ay',  # 투구 가속도 y축
    'az',  # 투구 가속도 z축
    'effective_speed',  # 타자가 느끼는 실질적인 공의 속도
    'release_spin_rate',  # 릴리스 순간의 회전수
    'release_extension',  # 릴리스 지점과 홈플레이트 사이 거리
    'spin_axis',  # 공의 회전축 (구질의 방향성)
    'arm_angle',  # 투수 팔의 릴리스 각도
    'age_pit',  # 투수 나이
    'pitcher_days_since_prev_game',  # 투수의 마지막 등판 이후 휴식일 수
    'api_break_z_with_gravity',  # 중력 포함 투구 낙폭 (낙차)
    'api_break_x_arm',  # 투수 기준 좌우 변화량
    'p_throws_L',  # 투수가 왼손잡이인지 여부 (One-hot)
    'p_throws_R',  # 투수가 오른손잡이인지 여부 (One-hot)
    'game_type_R',  # 경기 타입이 정규시즌(R)인지 여부 (정규 메타정보)
    'n_thruorder_pitcher',  # 해당 투수가 타자 라인업을 몇 번째 도는 중인지
    'if_fielding_alignment_Infield shade',  # 내야 수비 셰이드 전략 여부
    'if_fielding_alignment_Standard',  # 내야 수비 스탠다드 여부
    'if_fielding_alignment_Strategic',  # 내야 수비 전략적 배치 여부
    'of_fielding_alignment_4th outfielder',  # 외야 4인 배치 여부
    'of_fielding_alignment_Standard',  # 외야 수비 스탠다드 여부
    'of_fielding_alignment_Strategic'  # 외야 수비 전략적 배치 여부
]

swing_features = [
    'plate_x',  # 공이 홈플레이트 기준 좌우 어디를 통과했는지
    'plate_z',  # 공이 홈플레이트 기준 어느 높이를 통과했는지
    'balls',  # 현재 볼 카운트
    'strikes',  # 현재 스트라이크 카운트
    'outs_when_up',  # 타석 시점 아웃 카운트
    'bat_score_diff',  # 타석 시점 점수차 (타자 팀 기준)
    'api_break_x_batter_in',  # 타자 기준으로 몸쪽으로 휘는 정도
    'age_bat',  # 타자 나이
    'batter_days_since_prev_game',  # 타자의 마지막 출전 이후 휴식일 수
    'stand_L',  # 타자가 왼손잡이인지 여부 (One-hot)
    'stand_R',  # 타자가 오른손잡이인지 여부 (One-hot)
    'at_bat_number',  # 이 경기에서의 타석 번호
    'pitch_number',  # 이 타석 내에서의 투구 번호
    'n_priorpa_thisgame_player_at_bat',  # 이 경기에서의 타자의 이전 타석 수
    'pitch_type',  # 투구 타입 코드 (ex. 'FF' 등)
    'zone',  # 스트라이크 존 번호 (1~14)
    'type',  # 결과 타입 (B: 볼, S: 스트라이크, X: 인플레이)
    'pitch_name',  # 구질 명칭 (ex. 4-Seam Fastball)
    'sz_top',  # 타자 기준 스트라이크 존 상단
    'sz_bot'   # 타자 기준 스트라이크 존 하단
]



In [ ]:
print("pitch_features 개수:", len(pitch_features))
print("swing_features 개수:", len(swing_features))

total_features = len(pitch_features) + len(swing_features)
print("전체 feature 수:", total_features)

pitch_features 개수: 31
swing_features 개수: 20
전체 feature 수: 51


In [ ]:
# if fv_a, fv_b 에 feature 가 누락되어 모자란 경우 알림

In [ ]:
def filter_for_prediction(df: pd.DataFrame, mode: str, pitch_features, swing_features):
    assert mode in ['home', 'away'], "mode must be 'home' or 'away'"

    all_columns = df.columns.tolist()

    # 각 feature에 대해 top/bot suffix 포함 버전 생성
    pitch_top = [f"{col}_top" for col in pitch_features]
    pitch_bot = [f"{col}_bot" for col in pitch_features]
    swing_top = [f"{col}_top" for col in swing_features]
    swing_bot = [f"{col}_bot" for col in swing_features]

    # 홈 팀 기준: pitch_top + swing_bot
    if mode == 'home':
        keep_cols = pitch_top + swing_bot
    # 어웨이 팀 기준: pitch_bot + swing_top
    else:
        keep_cols = pitch_bot + swing_top

    # 실제 존재하는 컬럼만 유지 (예외 방지)
    keep_cols = [col for col in keep_cols if col in all_columns]

    return df[keep_cols].copy()

# pitch / swing features (이전에 정의된 리스트 사용)
# → suffix 없이 base 컬럼명 리스트여야 함
X_fv_home = filter_for_prediction(X_fv, mode='home', pitch_features=pitch_features, swing_features=swing_features)
X_fv_away = filter_for_prediction(X_fv, mode='away', pitch_features=pitch_features, swing_features=swing_features)


In [ ]:
X_fv_home, X_fv_away

(     release_speed_top  release_pos_x_top  ...  sz_top_bot  sz_bot_bot
 0            88.551534          -0.778282  ...    3.374426    1.609262
 1            90.999231          -1.930923  ...    3.396846    1.568188
 2            85.625628           0.521960  ...    3.471600    1.593280
 3            92.523392          -0.564035  ...    3.488129    1.603597
 4            89.312214           1.845725  ...    3.342484    1.555096
 ..                 ...                ...  ...         ...         ...
 196          88.073494          -0.800181  ...    3.494015    1.578686
 197          91.198131          -1.633551  ...    3.398299    1.537007
 198          92.804000          -1.659400  ...    3.476303    1.633091
 199          93.926119          -1.652836  ...    3.387754    1.586522
 200          89.192638          -1.450184  ...    3.427009    1.628120
 
 [201 rows x 51 columns],
      release_speed_bot  release_pos_x_bot  ...  sz_top_top  sz_bot_top
 0            90.629508          -0.

In [ ]:
# feature vector (201, 102)
X_fv_combined = pd.concat([X_fv_home, X_fv_away], axis=1)
X_fv_combined

,release_speed_top,release_pos_x_top,...,sz_top_top,sz_bot_top
0,88.551534,-0.778282,...,3.408528,1.623067
1,90.999231,-1.930923,...,3.454692,1.568846
2,85.625628,0.521960,...,3.326432,1.555075
3,92.523392,-0.564035,...,3.454094,1.553626
4,89.312214,1.845725,...,3.350992,1.559542
...,...,...,...,...,...
196,88.073494,-0.800181,...,3.456687,1.631084
197,91.198131,-1.633551,...,3.480841,1.594953
198,92.804000,-1.659400,...,3.514600,1.660467
199,93.926119,-1.652836,...,3.325299,1.566343


In [ ]:
# label 컬럼 추가
meta_fv_with_label = meta_fv.copy()
meta_fv_with_label['label'] = y_fv.values  # Series → 컬럼으로

# 예측용 feature와 함께 concat (index 맞춰서)
df_match = pd.concat([meta_fv_with_label, X_fv], axis=1)

# parquet 저장
df_match.to_parquet("statcast_20250601-20250619_match.parquet", index=False)


In [ ]:
meta_fv_with_label.columns.to_list()


['inning',
 'inning_topbot',
 'home_team',
 'away_team',
 'game_date',
 'batter',
 'pitcher',
 'post_home_score',
 'post_away_score',
 'home_score',
 'away_score',
 'events',
 'fv_mode',
 'label']

In [ ]:
X_fv_with_meta.to_parquet("statcast_20250601-20250619_match.parquet", index=False)